In [27]:
# import libraries
import os
import psycopg2
import pandas as pd

import boto3
from io import StringIO

from dotenv import load_dotenv
load_dotenv()  # only for local testing


In [2]:
sales_data = pd.read_csv("../data/2019-Oct.csv")
sales_data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [3]:
sales_data.shape

(42448764, 9)

In [ ]:
# split the original sales data into ten batches

chunk_size = round(sales_data.shape[0]/10)
chunk_size

In [45]:
i = 0
j = i + chunk_size
print(i, j)

sales_batch_1 = sales_data[i:j]
sales_batch_1

0 4244876


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...,...,...,...,...,...
4244871,2019-10-04 11:37:09 UTC,view,1004748,2053013555631882655,electronics.smartphone,huawei,130.50,514913214,5b2250a1-aa59-409b-9ea6-f986faaaaa85
4244872,2019-10-04 11:37:09 UTC,view,9300099,2053013554524586339,NaN,sony,579.14,535977220,1dff7997-04fe-4337-a6dd-f40f4341eee1
4244873,2019-10-04 11:37:09 UTC,view,1004042,2053013555631882655,electronics.smartphone,xiaomi,141.32,515891980,15c5c81d-a7cc-4e92-b3d3-0e93f11885db
4244874,2019-10-04 11:37:09 UTC,view,1200957,2172371436436455782,electronics.tablet,samsung,115.32,555283965,5b567ab8-7c9b-4d36-8c80-45b0a1671be7


In [38]:
sales_batch_1.shape

(4244876, 9)

In [46]:
i = j
j = i + chunk_size
print(i, j)

sales_batch_2 = sales_data[i:j]
sales_batch_2

4244876 8489752


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
4244876,2019-10-04 11:37:09 UTC,view,1004670,2053013555631882655,electronics.smartphone,samsung,1286.75,550750306,a1097cc2-f565-4b06-9647-016f5d4e6370
4244877,2019-10-04 11:37:09 UTC,view,9101349,2053013552888807671,computers.peripherals.mouse,steelseries,133.59,513842243,d0c5b73a-6427-4dc8-b69b-7264d450a411
4244878,2019-10-04 11:37:09 UTC,view,10100202,2053013553283072273,NaN,NaN,9.24,550970418,168a8a7c-dcf3-49cd-9bbb-16fdf8a86859
4244879,2019-10-04 11:37:09 UTC,view,1002633,2053013555631882655,electronics.smartphone,apple,358.57,512909450,67e25ac7-6aea-45a2-978a-019f0636da5b
4244880,2019-10-04 11:37:09 UTC,view,3700851,2053013565983425517,appliances.environment.vacuum,vitek,61.75,556699358,00ee697b-788f-4acb-8375-2f138e383861
...,...,...,...,...,...,...,...,...,...
8489747,2019-10-07 15:41:14 UTC,view,3200236,2053013555321504139,appliances.kitchen.meat_grinder,NaN,95.60,517138532,b4681185-1491-4543-b4e2-ca1921fefca5
8489748,2019-10-07 15:41:14 UTC,view,12703015,2053013553559896355,NaN,cordiant,42.99,518077563,010ab38a-913a-45e9-9810-ad9e3340ebca
8489749,2019-10-07 15:41:14 UTC,view,1801830,2053013554415534427,electronics.video.tv,yasin,158.05,556089522,36416358-95e2-4ed1-8db9-f88e7cb6e239
8489750,2019-10-07 15:41:14 UTC,view,32801045,2055156924273394455,NaN,kolibri,514.79,521206954,bc1d464e-2b91-4e34-8b42-524c87a4d156


In [47]:
# connect to s3

def connect_to_s3(aws_access_key_id, aws_secret_access_key):
    """Methods that connects to s3"""

    s3_client = boto3.client(
        "s3",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key
    )

    return s3_client


def df_to_s3(df, key, s3_bucket, aws_access_key_id, aws_secret_access_key):
    """Function that writes a data frame as a .csv file to a s3 bucket"""

    csv_buffer = StringIO()  # create buffer to temporarily store the Data Frame

    df.to_csv(csv_buffer, index=False)  # code to write the data frame as csv file

    s3_client = connect_to_s3(aws_access_key_id, aws_secret_access_key)

    s3_client.put_object(
        Bucket=s3_bucket, Key=key, Body=csv_buffer.getvalue()
    )  # this code writes the temp stored csv file and writes to s3

    print(f"The transformed data is saved as CSV in the following location s3://{s3_bucket}/{key}")

In [48]:
aws_access_key_id = os.getenv("aws_access_key_id")
aws_secret_access_key_id = os.getenv("aws_secret_access_key_id")
key = "kaggle-data-dump/Oct_data.csv"
s3_bucket = "waia-data-dump"

df_to_s3(sales_batch_1, key, s3_bucket, aws_access_key_id, aws_secret_access_key_id)

The transformed data is saved as CSV in the following location s3://waia-data-dump/kaggle-data-dump/Oct_data.csv


In [25]:
# connect to redshift

def connect_to_redshift(dbname, host, port, user, password):
    """Method that connects to redshift. This gives a warning so will look for another solution"""

    connect = psycopg2.connect(
        dbname=dbname, host=host, port=port, user=user, password=password
    )

    print("connection to redshift made")

    return connect


In [29]:
dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

connect = connect_to_redshift(dbname, host, port, user, password)

connection to redshift made


In [60]:
# connect to redshift and creating a table

cursor = connect.cursor()

query = f"""
CREATE TABLE bootcamp1.oct_sales ( 
            event_time DATETIME, 
            event_type VARCHAR(10), 
            product_id VARCHAR(20), 
            category_id VARCHAR(20), 
            category_code VARCHAR(100), 
            brand VARCHAR(50), 
            price FLOAT, 
            user_id INT, 
            user_session VARCHAR(50));
"""

    
cursor.execute(query)
connect.commit()

In [59]:
def load_from_s3_to_redshift(dbname, host, port, user, password, table_destination, s3_bucket, key, aws_access_key_id, aws_secret_access_key_id):
    
    connect = connect_to_redshift(dbname, host, port, user, password)
    cursor = connect.cursor()

    query = f"""
        COPY
        {table_destination}
        FROM
        's3://{s3_bucket}/{key}'
        with credentials
            'aws_access_key_id={aws_access_key_id};aws_secret_access_key={aws_secret_access_key_id}'
        FORMAT AS CSV
        IGNOREHEADER 1
        ;"""
    
    cursor.execute(query)
    connect.commit()

In [ ]:
# code to split the dataset into N number of datasets and 

chunk_size = round(sales_data.shape[0]/100)
i = 0
j = 0

while j <= sales_data.shape[0]:
    j = i + chunk_size
    print(i, j)
    subset_sales_data = sales_data[i:j]
    
    key = f"kaggle-data-dump/oct_data_{i}_to_{j-1}.csv"
    
    # load to s3
    df_to_s3(subset_sales_data, key, s3_bucket, aws_access_key_id, aws_secret_access_key_id)
    
    table_destination = "bootcamp1.oct_sales"
    # load to redshift
    load_from_s3_to_redshift(dbname, host, port, user, password, table_destination, s3_bucket, key, aws_access_key_id, aws_secret_access_key_id)
    
    print(f"loaded rows {i}_to_{j-1} to redshift")
    i = j 

0 424488
The transformed data is saved as CSV in the following location s3://waia-data-dump/kaggle-data-dump/oct_data_0_to_424487.csv
connection to redshift made
loaded rows 0_to_424487 to redshift
424488 848976
The transformed data is saved as CSV in the following location s3://waia-data-dump/kaggle-data-dump/oct_data_424488_to_848975.csv
connection to redshift made
loaded rows 424488_to_848975 to redshift
848976 1273464
The transformed data is saved as CSV in the following location s3://waia-data-dump/kaggle-data-dump/oct_data_848976_to_1273463.csv
connection to redshift made
loaded rows 848976_to_1273463 to redshift
1273464 1697952
The transformed data is saved as CSV in the following location s3://waia-data-dump/kaggle-data-dump/oct_data_1273464_to_1697951.csv
connection to redshift made
loaded rows 1273464_to_1697951 to redshift
1697952 2122440
The transformed data is saved as CSV in the following location s3://waia-data-dump/kaggle-data-dump/oct_data_1697952_to_2122439.csv
connec